In [12]:
import logging
import re
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import punkt
from gensim import corpora, models, similarities
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from collections import Counter
import re
import os
path = '/root/topic_model/'
directory = 'ap'
if not os.path.exists(directory):
    os.makedirs(directory)
file_path = '/root/topic_model/ap'
f = open('/root/topic_model/ap.txt','r')
remove_list = ['<DOC>', '<TEXT>' , '</DOC>' , '</TEXT>']
regex = re.compile(r"<DOCNO> *.* </DOCNO>")
lst = []
for line in f:
    for word in remove_list:
        if word in line  :
            line = line.replace(word,'')
    if regex.match(line):
        line = line.replace(line,'')
    if line.rstrip() in ['','\n']:
        line = line.replace(line,'')
    lst.append(line)
lst = [i for i in lst if i != '']
f.close()
for i in range(len(lst)):
    f = open(file_path + os.sep + str(i) + ".txt",'w')
    f.write(lst[i])  
f.close()

In [13]:
file_list = ((os.listdir(file_path)))
#print file_list
#filtered_dir = [i for i in directories if re.match(r'*txt', i)]
#print filtered_dir[0]
#print file_list
#print filtered_dir
#print directories
#print file_list[0]'''
#print path+filtered_dir[0]+os.sep+file_list[0][21]
print len(file_list)

2248


In [14]:
#Reading the incoming file as string.
data_list= []
for i in range(len(file_list)):
    data = []
    with open(file_path+os.sep+file_list[i], 'r') as myfile:
            data.append(myfile.read().lower().replace('\n', ''))
    data_list.append(data)
#print len(data_list)

In [15]:
scrubbed_final = []
sample_list = data_list[20:22]
for i in range(len(data_list)):   
#for i in range(len(sample_list)):
    print i
    #Removing Non-Ascii Characters from the data
    def removing_non_ascii(data):
        scrubbed = re.sub(r'[^\w]', ' ',data)
        return scrubbed
    data_scrubbed = removing_non_ascii(data_list[i][0])
    #data_scrubbed = removing_non_ascii(sample_list[i][0])
    tokenized_data = word_tokenize(data_scrubbed)
    scrubbed = [word for word in tokenized_data if word not in stop_words]
    word_freq = dict(Counter(scrubbed))
    final_word_list = []
    for i in range(len(scrubbed)):
        #if word_freq[scrubbed[i]] > 1 => Before
        if word_freq[scrubbed[i]] > 1 and len(scrubbed[i]) > 4  :
            #print scrubbed[i]
            final_word_list.append(scrubbed[i])
    scrubbed_final.append(final_word_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
#print len(scrubbed_final)
#print scrubbed_final[0:2]

In [17]:
dictionary = corpora.Dictionary(scrubbed_final)
#dictionary.save('/tmp/deerwester.dict')
#print dictionary
dictionary.compactify()
#corpus = [dictionary.doc2bow(text) for text in scrubbed]
#print corpus
#print(dictionary.token2id)
# Every scrubbed text should be passed here.
#new_doc = "consider consider consider random random Wiegerinck Wiegerinck Wiegerinck networks networks  networks networks networks"
#new_vec = dictionary.doc2bow(new_doc.split())

corpus = [dictionary.doc2bow(text) for text in scrubbed_final]


In [18]:
corpora.MmCorpus.serialize(path + os.sep + 'corpus.mm', corpus)
corpus_load = corpora.MmCorpus.serialize(path + os.sep + 'corpus.mm', corpus)
#corpus1 = corpora.MmCorpus('D:\\nips12\\corpus_trial.mm')

In [19]:
tfidf = models.TfidfModel(corpus)

In [20]:
corpus_tfidf = tfidf[corpus]
#for doc in corpus_tfidf:
#   print(doc)

In [17]:
#print corpus_tfidf[1]

[(279, 0.07393407716621142), (526, 0.11715377838227549), (629, 0.4608800835956926), (657, 0.0916003976584494), (694, 0.13122809192054205), (937, 0.130457297283664), (1046, 0.17651328189040122), (1189, 0.1754697961344172), (1399, 0.13122809192054205), (1473, 0.08991047323062479), (1662, 0.1754697961344172), (1935, 0.13122809192054205), (1972, 0.11169425865263713), (2267, 0.13473282146039067), (2638, 0.10975505979725371), (3054, 0.13473282146039067), (3137, 0.11390027502798124), (3706, 0.08200604312012744), (3830, 0.1754697961344172), (3869, 0.1754697961344172), (4056, 0.11333812422280407), (4098, 0.3824454440958693), (4162, 0.07393407716621142), (4831, 0.06328088735739655), (5610, 0.17040455516304837), (6218, 0.15049203447128026), (6443, 0.16515307803396323), (6940, 0.07193404215194214), (6992, 0.07527977986583266), (7125, 0.08276397125249482), (7352, 0.17846041267425158), (7392, 0.13887803669111817), (7849, 0.11243294409085883), (7900, 0.08563535236049627), (8044, 0.07473190423562591),

In [67]:
#print corpus
#print tfidf

In [25]:
LDA_Model2 = models.LdaModel(corpus, id2word=dictionary, num_topics=100, update_every= 4,passes=50)

In [59]:
print LDA_Model2.print_topics(30)


[(74, u'0.017*orion + 0.014*united + 0.011*icahn + 0.011*shuster + 0.010*troops + 0.009*texaco + 0.009*returns + 0.009*service + 0.009*eastern + 0.009*soldiers'), (0, u'0.038*would + 0.013*president + 0.011*chamorro + 0.011*saudi + 0.010*people + 0.009*defense + 0.009*report + 0.009*arabia + 0.008*administration + 0.008*nicaragua'), (52, u'0.029*people + 0.013*damage + 0.013*earthquake + 0.012*quake + 0.010*miles + 0.009*scale + 0.009*water + 0.009*richter + 0.008*homeless + 0.007*weather'), (78, u'0.020*nordstrom + 0.020*years + 0.013*symphony + 0.010*world + 0.010*committee + 0.009*orchestra + 0.009*fleisher + 0.008*would + 0.007*store + 0.007*state'), (98, u'0.029*greyhound + 0.025*champion + 0.022*george + 0.021*skull + 0.019*bones + 0.018*records + 0.017*record + 0.016*society + 0.015*breeze + 0.015*drivers'), (22, u'0.013*shuttle + 0.012*would + 0.011*monday + 0.011*government + 0.010*aspirin + 0.009*power + 0.009*space + 0.008*silverado + 0.008*people + 0.008*miners'), (15, u'0.

In [5]:
import pyLDAvis
import graphlab as gl

In [2]:
from __future__ import division
import graphlab as gl

ImportError: No module named prepare

In [26]:
import graphlab as gl
import pyLDAvis

In [31]:
import gensim
if not gensim.matutils.ismatrix(corpus):
    corpus_csc = gensim.matutils.corpus2csc(corpus)
else:
    corpus_csc = corpus
vocab = list(dictionary.token2id.keys())

In [32]:
print vocab

[u'yellow', u'revelers', u'woods', u'woody', u'cyprus', u'counterintelligence', u'aggression', u'looking', u'midafternoon', u'eligible', u'electricity', u'factoring', u'unanswered', u'refunding', u'demoted', u'hermann', u'meadows', u'placed', u'sinking', u'digit', u'hormone', u'regional', u'handling', u'microbe', u'stabbed', u'screaming', u'elevations', u'drollet', u'prize', u'broiler', u'sturgis', u'wednesday', u'circuitry', u'provided', u'commented', u'guardsmen', u'charter', u'tired', u'miller', u'pulse', u'holocaust', u'tires', u'second', u'rudman', u'gorman', u'jihan', u'resignations', u'theory', u'lukman', u'errors', u'centimeter', u'cooking', u'contributed', u'fingers', u'bonuses', u'updike', u'wertheim', u'increasing', u'admiral', u'shocks', u'specialist', u'dispatched', u'crouch', u'avert', u'reporter', u'avery', u'zlotys', u'error', u'affiliates', u'reported', u'ching', u'china', u'auctioned', u'affiliated', u'chinn', u'natured', u'substance', u'unwillingness', u'climbed', u'

In [33]:
print len(vocab)

11144


In [34]:
print len(dictionary)

11144


In [35]:
import numpy as np
fnames_argsort = np.asarray(list(dictionary.token2id.values()), dtype=np.int_)

In [38]:
term_freqs = corpus_csc.sum(axis=1).A.ravel()[fnames_argsort]
term_freqs[term_freqs == 0] = 0.01

In [46]:
doc_lengths = corpus_csc.sum(axis=0).A.ravel()
print doc_lengths

[  62.  135.   71. ...,  163.   70.  128.]


In [41]:
print corpus_csc[0]

  (0, 1165)	10.0


In [44]:
print file_list[0]

598.txt


In [45]:
print term_freqs

[ 15.   2.  16. ...,   3.   9.   3.]


In [48]:
doc_topic_dists = None
if doc_topic_dists is None:
    gamma, _ = LDA_Model2.inference(corpus)
    doc_topic_dists = gamma / gamma.sum(axis=1)[:, None]

In [50]:
assert doc_topic_dists.shape[1] == LDA_Model2.num_topics, 'Document topics and number of topics do not match {} != {}'.format(doc_topic_dists.shape[0], topic_model.num_topics)


In [51]:
topic = LDA_Model2.state.get_lambda()

In [52]:
topic = topic / topic.sum(axis=1)[:, None]

In [53]:
topic_term_dists = topic[:, fnames_argsort]

In [56]:
assert topic_term_dists.shape[0] == doc_topic_dists.shape[1]
sample = {'topic_term_dists': topic_term_dists, 'doc_topic_dists': doc_topic_dists,
           'doc_lengths': doc_lengths, 'vocab': vocab, 'term_frequency': term_freqs}

In [57]:
import pyLDAvis
vis_data = pyLDAvis.prepare(**sample)

In [58]:
pyLDAvis.display(vis_data)